1. Preprocesar el documento: Tokenizar el documento en oraciones.
2. Vectorizar: Convertir oraciones o palabras en vectores. Esto podría hacerse usando TF-IDF, bolsa de palabras, etc.
3. Calcular similitud: Crear una matriz de similitud donde cada elemento representa la similitud entre dos oraciones o palabras.
4. Normalizar: asegurarse de que cada fila sume uno.
5. Construir grafo: Usando la matriz de similitud, crear un grafo donde los nodos representen oraciones o palabras y los bordes representen puntuaciones de similitud.
6. Puntuar nodos: Implementar el mecanismo de clasificación de TextRank para puntuar cada nodo.
7. Extraer resultados: Para resumir, seleccionar las oraciones con la mejor clasificación. Para la extracción de palabras clave, elegir las palabras mejor clasificadas.

In [91]:
import pandas as pd
import numpy as np
import textwrap
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [92]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\diabl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\diabl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [93]:
df = pd.read_csv('./12-resumen/data_larazon_publico_v2.csv')

In [94]:
df.head()

,Unnamed: 0,indi,cuerpo,titular
0,0,0,dos semanas después de su puesta de largo y pr...,el submarino s-80 ya flota
1,1,1,"este viernes, el presidente del gobierno, pedr...",calviño y calvo alaban (sin darse cuenta) la g...
2,2,2,"el ministro del interior, fernando grande-marl...",el geo de la policía tendrá una nueva sede en ...
3,3,3,son días muy duros para la familia de olivia y...,"la madre de las niñas ""sobran las palabras par..."
4,4,4,sólo quedan 10 presos de eta por recibir los b...,sólo quedan 10 presos de eta por recibir el be...


In [95]:
doc = df["cuerpo"].sample()

In [96]:
print(doc.iloc[0])
print(len(doc.iloc[0]))

un subinspector y un agente de la policía nacional han sido detenidos en la tarde de este viernes en linares (jaén) cuando estaban fuera de servicio tras su presunta implicación en una agresión a un hombre en la calle esta misma jornada. según han informado fuentes de la policía a los medios, está previsto que los arrestados pasen a disposición judicial este sábado al juzgado de instrucción número 3 de la ciudad. en cualquier caso, ya se les ha abierto un expediente disciplinario por "estos lamentables hechos que no tienen que empañar la labor que muchos agentes realizan en las calles". el herido se encuentra ingresado en el hospital san agustín de linares con lesiones en la nariz y la cornea, y también ha resultado herida, aunque con menor gravedad, su hija de 14 años. en el material audiovisual difundido sobre la pelea por redes sociales se ve cómo varios hombres empiezan a dar golpes y empujones en plena calle a otro hombre, que acaba cayendo al suelo, siendo víctima de varios golpe

In [97]:
def wrap(x):
    return textwrap.fill(x, replace_whitespace=False, fix_sentence_endings=True)

In [98]:
print(wrap(doc.iloc[0]))
doc2 = wrap(doc.iloc[0])

un subinspector y un agente de la policía nacional han sido detenidos
en la tarde de este viernes en linares (jaén) cuando estaban fuera de
servicio tras su presunta implicación en una agresión a un hombre en
la calle esta misma jornada.  según han informado fuentes de la
policía a los medios, está previsto que los arrestados pasen a
disposición judicial este sábado al juzgado de instrucción número 3 de
la ciudad.  en cualquier caso, ya se les ha abierto un expediente
disciplinario por "estos lamentables hechos que no tienen que empañar
la labor que muchos agentes realizan en las calles". el herido se
encuentra ingresado en el hospital san agustín de linares con lesiones
en la nariz y la cornea, y también ha resultado herida, aunque con
menor gravedad, su hija de 14 años.  en el material audiovisual
difundido sobre la pelea por redes sociales se ve cómo varios hombres
empiezan a dar golpes y empujones en plena calle a otro hombre, que
acaba cayendo al suelo, siendo víctima de varios go

In [99]:
lineas = doc2.split(". ")
lineas

['un subinspector y un agente de la policía nacional han sido detenidos\nen la tarde de este viernes en linares (jaén) cuando estaban fuera de\nservicio tras su presunta implicación en una agresión a un hombre en\nla calle esta misma jornada',
 ' según han informado fuentes de la\npolicía a los medios, está previsto que los arrestados pasen a\ndisposición judicial este sábado al juzgado de instrucción número 3 de\nla ciudad',
 ' en cualquier caso, ya se les ha abierto un expediente\ndisciplinario por "estos lamentables hechos que no tienen que empañar\nla labor que muchos agentes realizan en las calles"',
 'el herido se\nencuentra ingresado en el hospital san agustín de linares con lesiones\nen la nariz y la cornea, y también ha resultado herida, aunque con\nmenor gravedad, su hija de 14 años',
 ' en el material audiovisual\ndifundido sobre la pelea por redes sociales se ve cómo varios hombres\nempiezan a dar golpes y empujones en plena calle a otro hombre, que\nacaba cayendo al suelo,

In [100]:
# Eliminamos de la lista de frases las frases vacias
lineas = [item for item in lineas if item.strip()]

In [101]:
len(lineas)

6

In [102]:
tokenizar = TfidfVectorizer(
    stop_words=stopwords.words("spanish"),
    norm="l1"  # Aquí indicamos que el peso va a ir de 0 a 1
)

In [103]:
X = tokenizar.fit_transform(lineas)
X

<6x114 sparse matrix of type '<class 'numpy.float64'>'
	with 122 stored elements in Compressed Sparse Row format>

In [104]:
X.shape

(6, 114)

In [105]:
# Cualculamos como se parecen cada frase con las otras frases
# Matriz de similitud
S = cosine_similarity(X)

In [106]:
S.shape

(6, 6)

In [107]:
print(S)

[[1.         0.04277624 0.         0.02793528 0.1136466  0.        ]
 [0.04277624 1.         0.         0.         0.         0.        ]
 [0.         0.         1.         0.         0.         0.04971578]
 [0.02793528 0.         0.         1.         0.01687202 0.        ]
 [0.1136466  0.         0.         0.01687202 1.         0.        ]
 [0.         0.         0.04971578 0.         0.         1.        ]]


- Normalizamos para asegurarnos de que cada fila sume 1
- Se suman todos los valores de cada fila y estos valores sumados de cada fila se van dividiendo entre cada elemento de esa fila dando una matriz normalizada
- Ejemplo:
S = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])

_________________________

S.sum(axis=1, keepdims=True)
array([[ 6],
        [15],
        [24]])

_________________________

S_normalized = S / S.sum(axis=1, keepdims=True)
array([[0.16666667, 0.33333333, 0.5       ],
        [0.26666667, 0.33333333, 0.4       ],
        [0.29166667, 0.33333333, 0.375     ]])


In [108]:
# Normalizamos para asegurarnos de que cada fila sume 1
# Se suman todos los valores de cada fila y estos valores sumados de cada fila se van dividiendo entre cada elemento de esa fila dando una matriz normalizada
S = S / S.sum(axis=1, keepdims=True)
S

array([[0.84433921, 0.03611766, 0.        , 0.02358685, 0.09595628,
        0.        ],
       [0.0410215 , 0.9589785 , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.95263882, 0.        , 0.        ,
        0.04736118],
       [0.02673725, 0.        , 0.        , 0.95711429, 0.01614845,
        0.        ],
       [0.10052608, 0.        , 0.        , 0.01492414, 0.88454978,
        0.        ],
       [0.        , 0.        , 0.04736118, 0.        , 0.        ,
        0.95263882]])

In [109]:
# Matriz de transición uniforme
U = np.ones_like(S)
U

array([[1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.]])

In [110]:
U.shape

(6, 6)

In [111]:
# Hacemos que cada fila tambien sume 1
U = np.ones_like(S)/len(S)
U

array([[0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.16666667],
       [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.16666667],
       [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.16666667],
       [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.16666667],
       [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.16666667],
       [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.16666667]])

In [112]:
# Matriz de similitud suavizada
factor = 0.15
S = (1 - factor) * S + factor * U # Con esto hacemos que la matriz S valga un 85% y la matriz U valga un 15%
S

array([[0.74268833, 0.05570001, 0.025     , 0.04504882, 0.10656284,
        0.025     ],
       [0.05986827, 0.84013173, 0.025     , 0.025     , 0.025     ,
        0.025     ],
       [0.025     , 0.025     , 0.834743  , 0.025     , 0.025     ,
        0.065257  ],
       [0.04772667, 0.025     , 0.025     , 0.83854715, 0.03872618,
        0.025     ],
       [0.11044716, 0.025     , 0.025     , 0.03768552, 0.77686731,
        0.025     ],
       [0.025     , 0.025     , 0.065257  , 0.025     , 0.025     ,
        0.834743  ]])

<img src="./12-resumen/Code_XysgwK2HQ9.png" style="width:400px;hight:auto"></img>

In [113]:
# Calcular la prob de en que frase se esta posicionado en el tiempo
eigenvals, eigenvecs = np.linalg.eig(S.T)

In [114]:
eigenvals

array([1.        , 0.64527545, 0.85      , 0.78734677, 0.81561229,
       0.76948599])

In [115]:
eigenvecs[:,0]

array([-0.4221828 , -0.40130979, -0.40818954, -0.40152309, -0.40774249,
       -0.40818954])

In [116]:
scores = eigenvecs[:,0]

In [117]:
sort_idx = np.argsort(-scores)
sort_idx

array([1, 3, 4, 5, 2, 0], dtype=int64)

In [118]:
print("Crear Resumen:")
for i in sort_idx[:5]:
    print(wrap("%.2f: %s" % (scores[i], lineas[i])))

Crear Resumen:
-0.40:  según han informado fuentes de la
policía a los medios, está
previsto que los arrestados pasen a
disposición judicial este sábado
al juzgado de instrucción número 3 de
la ciudad
-0.40: el herido se
encuentra ingresado en el hospital san agustín de
linares con lesiones
en la nariz y la cornea, y también ha resultado
herida, aunque con
menor gravedad, su hija de 14 años
-0.41:  en el material audiovisual
difundido sobre la pelea por redes
sociales se ve cómo varios hombres
empiezan a dar golpes y empujones
en plena calle a otro hombre, que
acaba cayendo al suelo, siendo
víctima de varios golpes y contusiones.
desde el ayuntamiento de
linares han condenado "enérgicamente la
brutal agresión cometida esta
tarde" y exigen "que se actúe con
contundencia contra una actitud que
ensucia la imagen de un cuerpo que
siempre ha velado por la seguridad
de los linarenses"
-0.41: igualmente,
esperan que "se tomen las medidas disciplinarias y
de cualquier otra
índole que sean nece